In [59]:
import glob
import pandas as pd
import numpy as np
import re

# Project idea and description...


Tadatadatada

# Data aquisition

In the first ... 

We need to get all the following keywords from twitter :

abu sayyaf,
afghanistan,
agro,
al-qaeda,
al-qaeda in the arabian peninsula,
al-qaeda in the islamic maghreb,
al-shabaab,
ammonium nitrate,
attack,
biological weapon,
car bomb,
chemical weapon,
conventional weapon,
dirty bomb,
eco-terrorism,
environmental terrorism,
euskadi ta askatasuna,
extremism,
farc,
fundamentalism,
hamas,
hezbollah,
improvised explosive device,
iran,
iraq,
irish republican army,
islamist,
jihad,
nationalism,
nigeria,
nuclear,
nuclear enrichment,
pakistan,
palestine liberation front,
pirates,
plo,
political radicalism,
recruitment,
somalia,
suicide attack,
suicide bomber,
taliban,
tamil tigers,
tehrik-i-taliban pakistan,
terror,
terrorism,
weapons-grade,
yemen,

## Analysis of the tweets

### Interrupted time series with regression
We will now focus on getting an understanding of the tweet distribution over time, and how the massive revelations of online surveillance in June 2013 might have caused a chilling effect. We will follow the paper's original way of doing the interrupted time series, with regression analysis.

First, let's see what those big files yield by reading one. We need to first find all the archives.

In [42]:
archive_pathnames = glob.glob('./data/*.gz')
print(f"Found {len(archive_pathnames)} archives")

Found 44 archives


Now, what is in the first archive ?

In [46]:
df = pd.read_csv(archive_pathnames[0])
print(df.shape)
df.head()

(9299, 22)


,id,conversation_id,created_at,date,time,user_id,username,name,tweet,language,...,photos,replies_count,retweets_count,likes_count,hashtags,link,retweet,video,thumbnail,reply_to
0,506278868614148097,506278868614148097,2014-09-01 05:14:23 CEST,2014-09-01,05:14:23,8.343178e+07,starman1981,truckerstrong news,ISIS laptop reveals terror group were working ...,en,...,[],0,0.0,0.0,[],https://twitter.com/starman1981/status/5062788...,False,False,NaN,[]
1,506272708037214208,506272708037214208,2014-09-01 04:49:54 CEST,2014-09-01,04:49:54,7.134575e+08,patriotsorg,Patriots Billboard,#SEIZED ISIS LAPTOP REVEALS THEY’RE DEVELOPING...,en,...,[],0,3.0,1.0,['seized'],https://twitter.com/PatriotsOrg/status/5062727...,False,False,NaN,[]
2,506263382459428864,506263382459428864,2014-09-01 04:12:50 CEST,2014-09-01,04:12:50,2.630728e+08,mlgeek,My Little Geek,@bjorntipling Perhaps Swift is a biological we...,en,...,[],0,0.0,0.0,[],https://twitter.com/mlgeek/status/506263382459...,False,False,NaN,[]
3,506261754197061632,506261754197061632,2014-09-01 04:06:22 CEST,2014-09-01,04:06:22,2.510451e+09,msgubot,Cédric Moro (backup),G+ #ebola USA created Ebola virus as biologica...,en,...,[],0,0.0,0.0,['ebola'],https://twitter.com/MSGUBOT/status/50626175419...,False,False,NaN,[]
4,506239829819281408,506239829819281408,2014-09-01 02:39:15 CEST,2014-09-01,02:39:15,7.134575e+08,patriotsorg,Patriots Billboard,#SEIZED ISIS LAPTOP REVEALS THEY’RE DEVELOPING...,en,...,[],1,2.0,1.0,['seized'],https://twitter.com/PatriotsOrg/status/5062398...,False,False,NaN,[]


We see a lot of information there. But what we are interested in is the user interactions around a topic that they might the government to track, for example. Therefore, we can count the number of tweets themselves, not any of their content or information, but also the number of likes and rerplies ! Each of these actions can make the user fear such surveillance. Retweets also, but because we also collect the retweets themselves, they are already there ! Therefore, we will first count the number of tweets per month, with the number of likes and retweets added as well.

In [58]:
# Keep only interesting columns and sum them all together by month. We set to parse dates so we can group by month
df = pd.read_csv(archive_pathnames[0], usecols=["date", "likes_count", "replies_count"], parse_dates=["date"], lineterminator='\n')
df["tweet_count"] = 1
grouped_df = df.set_index('date').groupby(pd.Grouper(freq='M')).sum()
grouped_df["user_interactions"] = grouped_df["likes_count"] +  grouped_df["tweet_count"] +  grouped_df["replies_count"]

grouped_df.head()

,replies_count,likes_count,tweet_count,user_interactions
date,,,,
2012-01-31,49,51,207,307
2012-02-29,33,27,177,237
2012-03-31,37,10,192,239
2012-04-30,40,32,156,228
2012-05-31,46,17,237,300


We will want to merge all twitter keyworkds into the same dataframe, therefore we will give the name of the keywork to the column where all values are summed, instead of just user_interactions. Let's extract the name from the archive path.

In [47]:
df = pd.read_csv(archive_pathnames[0], usecols=["date", "likes_count", "replies_count"], parse_dates=["date"], lineterminator='\n')
df["tweet_count"] = 1
grouped_df = df.set_index('date').groupby(pd.Grouper(freq='M')).sum()

df1 = pd.read_csv(archive_pathnames[3], usecols=["date", "likes_count", "replies_count"], parse_dates=["date"], lineterminator='\n')
df1["tweet_count"] = 1
grouped_df1 = df1.set_index('date').groupby(pd.Grouper(freq='M')).sum()

monthly_counts = pd.concat([grouped_df, grouped_df1], axis=1, keys=['1','2'])
monthly_counts

1                                     2              \
           replies_count likes_count tweet_count replies_count likes_count   
date                                                                         
2012-01-31            49          51         207         131.0        65.0   
2012-02-29            33          27         177         145.0       130.0   
2012-03-31            37          10         192          22.0        14.0   
2012-04-30            40          32         156          18.0        24.0   
2012-05-31            46          17         237          36.0        26.0   
2012-06-30            50          39         237          22.0        10.0   
2012-07-31            57          89         227          15.0        20.0   
2012-08-31            45          35         263          29.0        18.0   
2012-09-30            39          14         192          32.0        78.0   
2012-10-31            36         146         215         179.0       106.0   
2012-11-30            50         101         237         125.0       114.0   
2012-12-31            74         269         226          20.0        17.0   
2013-01-31            55          50         221          60.0        44.0   
2013-02-28            60         239         171          65.0        64.0   
2013-03-31            40         100         201          15.0        17.0   
2013-04-30           126         313        1463          19.0        18.0   
2013-05-31            70         125         379          23.0        35.0   
2013-06-30            45          69         266          36.0        33.0   
2013-07-31            41          66         258           7.0         6.0   
2013-08-31            61          52         263          42.0        44.0   
2013-09-30            75          58         242          81.0        92.0   
2013-10-31            31          46         232         115.0       147.0   
2013-11-30            37          74         154         307.0       605.0   
2013-12-31            61         101         187         139.0       242.0   
2014-01-31            40          77         220         159.0       338.0   
2014-02-28            47          96         399          24.0        40.0   
2014-03-31            63          95         180          21.0        57.0   
2014-04-30            48         517         161           5.0         6.0   
2014-05-31            32          61         202          20.0        33.0   
2014-06-30            54          53         191          24.0        52.0   
2014-07-31            31          60         243          60.0       202.0   
2014-08-31           137         156        1075          24.0       113.0   
2014-09-30             2           3           7           NaN         NaN   

                        
           tweet_count  
date                    
2012-01-31      1419.0  
2012-02-29      1838.0  
2012-03-31       835.0  
2012-04-30       409.0  
2012-05-31      1223.0  
2012-06-30       593.0  
2012-07-31       320.0  
2012-08-31       395.0  
2012-09-30       689.0  
2012-10-31      1122.0  
2012-11-30      1748.0  
2012-12-31       729.0  
2013-01-31       775.0  
2013-02-28       692.0  
2013-03-31       100.0  
2013-04-30       764.0  
2013-05-31       228.0  
2013-06-30       283.0  
2013-07-31        47.0  
2013-08-31       346.0  
2013-09-30       332.0  
2013-10-31      1011.0  
2013-11-30      2847.0  
2013-12-31      1170.0  
2014-01-31      2177.0  
2014-02-28       333.0  
2014-03-31       284.0  
2014-04-30        58.0  
2014-05-31       301.0  
2014-06-30       394.0  
2014-07-31       738.0  
2014-08-31       393.0  
2014-09-30         NaN

In [31]:
# print(pd.read_csv(archive_pathnames[0], usecols=["date", "likes_count"]))
total_tweet_nb = 0
monthly_counts = pd.DataFrame([])

for archive_pathname in archive_pathnames:
    print(f"Reading {archive_pathname} file")
    df = pd.read_csv(archive_pathname, usecols=["date", "likes_count"], parse_dates=["date"], lineterminator='\n')
    total_tweet_nb = total_tweet_nb + df.size
    print(f"Shape is : {df.shape} \n")
    print(f"We have {np.round(total_tweet_nb/1000000, 2)} million tweets as of the latest file")
    
    df["tweet_count"] = 1
    df = df.set_index('date').groupby(pd.Grouper(freq='M')).sum()
    monthly_counts = pd.merge(monthly_counts df)
    
print(f"We have {np.round(total_tweet_nb/1000000, 2)} million tweets !")

Reading ./data/biological weapon_full.gz file
Shape is : (9281, 2) 



NameError: name 'np' is not defined

In [1]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()


ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).